※ Threadの実行結果を確認するためにbashのコマンド実行でrubyを動かしてます

# Threadクラス

* Threadクラス
    * スレッドを表すクラス
    * 並行プログラミングが可能となる
    * Ruby1.9よりネイティブスレッドを用いて実装されている
        * 同時に実行されるネイティブスレッドは常に一つ
            * Giant VM lockを有しているため
    * メインスレッド
        * プログラムの開始と同時に生成されるスレッド
        * これが終了すると他のスレッドも含めてプログラムが終了する
    * カレントスレッド
        * 現在実行中のスレッド
    * スレッドの実行はスケジューリングされている
        * 優先順位付きのラウンドロビン・スケジューリングという方式で管理

## スレッドの生成

* メソッド
    * Thread.new
        * スレッドを生成する
        * initializeメソッドが呼ばれる
    * Thread.start, Thread.fork
        * スレッドを生成する
        * initializeメソッドは呼ばれない

In [6]:
%%bash
name=thread

### ソースの編集
cat <<- EOS > /tmp/${name}.rb

p t = Thread.new{ sleep 1}
p t = Thread.new(3){|t| sleep t}
EOS

### 実行
ruby /tmp/${name}.rb

#<Thread:0x007fd02ca5a038@/tmp/thread.rb:2 run>
#<Thread:0x007fd02ca59d40@/tmp/thread.rb:3 run>


## スレッドの状態

* メソッド
    * status
        * 生成したスレッドの実行状態を保持
    * alive?
        * スレッドが生きているかを確認
    * stop?
        * スレッドが終了、もしくは停止していることを確認
        
* 状態
    * run
        * 実行中、または実行可能状態
            * 生成直後
            * runメソッドやwakeupメソッドで起こされたスレッド
            * statusメソッドで確認するとrunが返る
    * sleep
        * 一時停止状態
            * Thread.stopメソッドやjoinメソッドにより一時停止されたスレッド
            * statusメソッドで確認するとsleepが返る
    * aborting
        * 終了処理中
            * killメソッドなどで終了されるスレッドは一時的にこの状態になる
            * statusメソッドで確認するとabortingが返る
    * dead
        * <font color="red">正常終了したスレッド</font>
            * killメソッドで終了したときや通常終了した状態
            * statusメソッドで確認すると<font color="red">false</font>が返る
        * <font color="red">異常終了したスレッド</font>
            * 例外など
            * statusメソッドで確認すると<font color="red">nil</font>が返る

In [19]:
%%bash
name=thread

### ソースの編集
cat <<- EOS > /tmp/${name}.rb

p t = Thread.new{ sleep 2}

puts "スレッドが生きている事の確認"
puts t.status
puts t.alive?
puts t.stop?
puts

sleep 1

puts "スレッドが停止している事の確認"
puts t.status
puts t.alive?
puts t.stop?
puts

sleep 2

puts "スレッドが終了している事の確認"
puts t.status
puts t.alive?
puts t.stop?

EOS

### 実行
ruby /tmp/${name}.rb

#<Thread:0x007fd69a859800@/tmp/thread.rb:2 run>
スレッドが生きている事の確認
sleep
true
true

スレッドが停止している事の確認
sleep
true
true

スレッドが終了している事の確認
false
false
true


## スレッドの再開、一時停止、終了

* メソッド
    * 再開
        * run
            * スレッドを再開する
        * wakeup
            * 実行可能状態にする
            * 実行されるかどうかは他のスレッドの状況による
    * 一時停止状態
        * Thread.stop
            * スレッドを停止している状態
        * join
            * 他のスレッドを待っている状態
    * 終了
        * kill, exit
            * スレッドを終了する
            * ensure節があれば実行する
        * kill!, exit!
            * スレッドを終了する
            * ensure節があっても実行されない
        * Thread.kill
            * 指定したスレッドのexitメソッドを呼び出す
        * Thread.exit
            * カレントスレッドのexitメソッドを呼び出す

In [12]:
%%bash
name=thread

### ソースの編集
cat <<- EOS > /tmp/${name}.rb
t = Thread.new do
  Thread.stop
  puts "OK\n"
end

### スレッドがsleep状態になるのを待つ
sleep 1

### スレッドの再開
puts t.run
EOS

### ruby実行
ruby /tmp/${name}.rb

OK
#<Thread:0x007fcef339eaf8>


In [23]:
%%bash
name=thread

### ソースの編集
cat <<- EOS > /tmp/${name}.rb
t = Thread.new do
  Thread.exit
end

### スレッドが終了するのを待つ
sleep 1

### スレッドの状態確認
p t
puts t.status
EOS

### ruby実行
ruby /tmp/${name}.rb

#<Thread:0x007f9f9620b740@/tmp/thread.rb:1 dead>
false


## スレッド終了時のensure節

* ensure節がある場合
    * スレッド終了時に実行される
    * 他のスレッドからkillで終了させられたときも実行する
    * kill!メソッドが呼び出されたときは実行されない
        * メインスレッドのみkill!メソッドが呼ばれた場合は各スレッドでは実行される

In [28]:
%%bash
name=thread

### ソースの編集
cat <<- EOS > /tmp/${name}.rb
t = Thread.new do
  begin
    sleep 10000
  ensure
    puts "Killed."
  end
end

### スレッドがsleepになるのを待つ
sleep 1

### スレッドをkillする
t.kill

EOS

### ruby実行
ruby /tmp/${name}.rb

Killed.


## スレッド中の例外

* スレッド中で例外発生
    * rescureで捕捉しなかった場合
        * 通常はそのスレッドのみが警告なしに終了される
        * joinメソッドで待っている他のスレッドがある場合は待っているスレッドに対して同じ例外が再度発生する
* 例外が発生した場合のプログラム自体の終了させ方
    * Thread.abort_on_exceptionメソッドにtrueを設定する
    * 特定のスレッドのabort_on_exceptionメソッドにtrueを設定する
    * グローバル変数`$DEBUG`にtrueを設定し、プログラムを-d付きオプションで実行する

In [39]:
%%bash
name=thread

### ソースの編集
cat <<- EOS > /tmp/${name}.rb

t = Thread.new {Thread.pass; raise "Raise exception"}

### tは異常終了
sleep 1
p t
p t.status

### eはtを待つ
e = Thread.new do
  begin
    p t.join
  rescue => ex
    p ex.message
  end
end

### pryで実行すると以下のようなメッセージが表示される
puts "Raise exception"

EOS

### ruby実行
ruby /tmp/${name}.rb

#<Thread:0x007fad921e3b28@/tmp/thread.rb:2 dead>
nil
Raise exception
